In [3]:
!pip install -U datasets fsspec huggingface_hub
!pip install vllm

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.32.3-py3-none-any.whl.metadata (14 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached hf_xet-1.1.2-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached huggingface_hub-0.32.3-py3-none-any.whl (512 kB)
Using cached hf_xet-1.1.2-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
    

In [5]:
ds["train"]

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

In [6]:
# from datasets import load_dataset

# ds = load_dataset("openai/gsm8k", "main")
# #dataset = load_dataset('squad', split='train')


#ds = load_dataset("openai/gsm8k", "socratic")

In [4]:
len(ds["train"]["question"])
len(ds["train"]["answer"])

7473

In [11]:
# print(ds["train"]["answer"][0])
# print(ds["train"]["answer"][0].split('\n#### ')[1])




# def process_ground_truth(ans):
#     return ans.split('\n#### ')[1]

# with ProcessPoolExecutor() as executor:
#     ground_truth = list(executor.map(process_ground_truth, ds["train"]["answer"]))
# len(ground_truth)



Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
72


7473

In [17]:
from datasets import load_dataset
from vllm import LLM, SamplingParams
from drgrpo_grader import r1_zero_reward_fn
from concurrent.futures import ProcessPoolExecutor
import os
def preprocess_dataset(ds):
    questions, answers =  ds["train"]["question"], ds["train"]["answer"]
    with open("prompts/r1_zero.prompt", "r", encoding="utf-8") as f:
        prompt_string = f.read()
    def process_question(q):
        return prompt_string.format(question=q)
    def process_ground_truth(ans):
    return ans.split('\n#### ')[1]

    with ProcessPoolExecutor() as executor:
        question_prompts = list(executor.map(process_question, ds["train"]["question"]))
    with ProcessPoolExecutor() as executor:
        ground_truth = list(executor.map(process_ground_truth, ds["train"]["answer"]))
    return question_prompts, ground_truth



ds = load_dataset("openai/gsm8k", "main")

question_prompts, ground_truth = preprocess_dataset(ds)


# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
temperature=1.0, top_p=1.0, max_tokens=1024, stop=["\n"]
)
# Create an LLM.

llm = LLM(model="Qwen/Qwen2.5-1.5B")

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
model_outputs = llm.generate(question_prompts, sampling_params)

# Print the outputs.
final_output = []
for output, gt in zip(model_outputs, ground_truth):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    res = r1_zero_reward_fn(generated_text, gt)
    
    final_output.append([res['format_reward'], res['answer_reward'], res['reward']])



'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: {question}\nAssistant: <think>'

In [5]:

#r1_zero_reward_fn(response, ground_truth, fast=True)
# Sample prompts.
prompts = [
"Hello, my name is",
"The president of the United States is",
"The capital of France is",
"The future of AI is",
]
# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
temperature=1.0, top_p=1.0, max_tokens=1024, stop=["\n"]
)
# Create an LLM.

llm = LLM(model="Qwen/Qwen2.5-1.5B")

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(ds["train"]["question"], sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

INFO 06-02 05:22:20 [__init__.py:247] No platform detected, vLLM is running on UnspecifiedPlatform


2025-06-02 05:22:21.462756: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


WARNING 06-02 05:22:26 [_custom_ops.py:21] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')
INFO 06-02 05:22:28 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-02 05:22:28 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-02 05:22:28 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


RuntimeError: Device string must not be empty